In [16]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (7,7) # Make the figures a bit bigger

import tensorflow as tf

from tensorflow.contrib.rnn.python.ops import rnn_cell

In [2]:
df = pd.read_csv('../data/will_play_text.csv', sep=';', header=None, names=['row', 'piece', 'chapter', 'section', 'character', 'line'])

In [3]:
sentences = df[~df.character.isnull()].line

# mapping
letters = ['>', '<'] + list(set(''.join(list(sentences))))
nletters = len(letters)

bw_mapping = dict(enumerate(letters, 1))
fw_mapping = dict([(c, ix) for ix, c in enumerate(letters, 1)])

rsen_sz = 64
sen_sz = 1 + rsen_sz + 1

def pad_sentence(s):
    l = len(s)
    coded = [fw_mapping[c] for c in ['>'] + list(s) + ['<']]
    return coded + [0] * (sen_sz - 1 - l)

# dataset
sentences = [pad_sentence(s) for s in sentences if len(s)<=rsen_sz]
sentences = np.array(sentences)
sentences.shape

(111005, 67)

In [4]:
sentences[2]

array([ 1, 14, 58, 41, 57, 35, 61, 51, 67, 63, 33, 51, 57, 66, 33, 60, 61,
       63, 37, 69,  7, 58, 41, 51, 41, 57, 67, 23, 23, 51, 58, 63, 66, 57,
       60, 17, 57, 58, 51, 69, 57, 35, 61, 60,  7, 70, 66,  2,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [17]:
hidden_state_size = 256

def rnn_text(sen_sz=sen_sz):
    cell_state = tf.placeholder(tf.float32, [batch_size, hidden_state_size])
    hidden_state = tf.placeholder(tf.float32, [batch_size, hidden_state_size])
    init_state = rnn_cell.LSTMStateTuple(cell_state, hidden_state)
    
    seq_input = tf.placeholder(tf.int64, [None, sen_sz])
    
    xs = seq_input[:, :-1]
    ys = seq_input[:, 1:]
    
    xs_valid_mask = tf.not_equal(xs, 0)    
    xs_seq_len = tf.reduce_sum(tf.to_int32(xs_valid_mask), axis=1)
    
    xs_one_hot = tf.one_hot(xs, depth=nletters)
    ys_one_hot = tf.one_hot(ys, depth=nletters)
    
    rnn_cell = rnn_cell.LSTMCell(hidden_state_size, state_is_tuple=True)
    outputs, state = tf.nn.dynamic_rnn(rnn_cell, xs_one_hot, sequence_length=xs_seq_len, initial_state=init_state)

    with tf.variable_scope("output"):
        W_hy = tf.get_variable('W_hy', [hidden_state_size, nletters])
        B_hy = tf.get_variable('B_hy', [nletters])
    
    logits = tf.tensordot(outputs, W_hy, axes=1) + B_hy
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=ys_one_hot, logits=logits)
    masked_cross_entropy = tf.multiply(cross_entropy, tf.to_float(xs_valid_mask))
    summed_entropy = tf.reduce_sum(masked_cross_entropy, 1)
    sequence_entropy = tf.divide(summed_entropy, tf.to_float(xs_seq_len))
    total_entropy = tf.reduce_mean(sequence_entropy)
    
    return seq_input, (cell_state, hidden_state), total_entropy

In [18]:
def my_optimize(sess, train, Xs, loss,
                name='train', steps=1000,
                optimizer=tf.train.GradientDescentOptimizer(0.5),
                dict={}, batch_size=100, report_steps=500):
    train_step = optimizer.minimize(loss)
    
    sess.run(tf.global_variables_initializer())

    for i in range(steps):
        
        # Such a hack
        start = (i * batch_size) % train.shape[0]
        end = ((i+1) * batch_size) % train.shape[0]
        
        if start < end:
            batch_xs = train[start:end]
            dict = dict.copy()# Avoid getting the keys from a different graph
            dict[Xs] = batch_xs
            _, xloss = sess.run([train_step, loss], feed_dict=dict)
            
            if i % report_steps == 0:
                print('Step {}\tLoss: {}'.format(i, xloss))

In [19]:
batch_size = 256

g = tf.Graph()
with g.as_default():
    with tf.Session() as sess:
        
        Xs, (cell_state, hidden_state), loss = rnn_text()
        
        optimizer = tf.train.AdamOptimizer()
        my_optimize(sess,
                    sentences,
                    Xs,
                    loss,
                    steps=31,
                    report_steps=500,
                    name='train-1',
                    batch_size=batch_size,
                    dict={cell_state: np.zeros((batch_size, hidden_state_size)),
                          hidden_state: np.zeros((batch_size, hidden_state_size)),
                         },
                    optimizer=optimizer)

UnboundLocalError: local variable 'rnn_cell' referenced before assignment

In [15]:
tf.__version__

'1.1.0'